# Food API

In [ ]:
import requests

# Define your product search term
product_name = "raisin"
search_url = "https://world.openfoodfacts.org/cgi/search.pl"

# Parameters for the API request
params = {
    "search_terms": product_name,
    "search_simple": 1,
    "json": 1,
    "page_size": 5,  # Limit the number of results for performance
}

# Make the request
response = requests.get(search_url, params=params)

# Process the results
if response.status_code == 200:
    data = response.json()
    if data.get("products"):
        print(f"Found {len(data['products'])} products matching '{product_name}':\n")
        print("Full JSON response for all products:\n")
        # Print the full JSON response for all matching products
        for product in data["products"]:
            print(product)  # Print the full product JSON
    else:
        print("No products found for your search term.")
else:
    print(f"Failed to retrieve data. HTTP Status code: {response.status_code}")


In [ ]:
import requests

# Define your product search term
product_name = "Muesli Raisin, Figue, Abricot"
search_url = "https://world.openfoodfacts.org/cgi/search.pl"

# Parameters for the API request
params = {
    "search_terms": product_name,
    "search_simple": 1,
    "json": 1,
    "page_size": 1,  # Limit results for simplicity
}

# Make the request
response = requests.get(search_url, params=params)

# Process the results
if response.status_code == 200:
    data = response.json()
    if data.get("products"):
        product = data["products"][0]  # Take the first product
        product_name = product.get("product_name", "Unknown Product")
        barcode = product.get("code", "N/A")
        carbon_footprint_100g = product.get("carbon_footprint_100g", None)
        carbon_footprint_debug = product.get("carbon_footprint_from_known_ingredients_debug", None)
        agribalyse_co2_total = product.get("agribalyse", {}).get("co2_total", None)

        print(f"Product Name: {product_name}")
        print(f"Barcode: {barcode}")
        
        # Display carbon footprint information
        if carbon_footprint_100g:
            print(f"Carbon Footprint (per 100g): {carbon_footprint_100g} g CO₂e")
        elif agribalyse_co2_total:
            print(f"Agribalyse Total CO₂ Emissions: {agribalyse_co2_total} kg CO₂e")
        elif carbon_footprint_debug:
            print(f"Estimated Carbon Footprint (from ingredients): {carbon_footprint_debug}")
        else:
            print("Carbon Footprint data is not available.")
    else:
        print("No products found for your search term.")
else:
    print(f"Failed to retrieve data. HTTP Status code: {response.status_code}")


In [ ]:
!pip install pandas

In [ ]:
# Load Agribalyse data
import pandas as pd

file_path = '../data/agribalyse-31-detail-par-ingredient.csv'
agribalyse_data = pd.read_csv(file_path)

# Search for a product by name or category
search_term = "Muesli"
matched_products = agribalyse_data[agribalyse_data["Nom Français"].str.contains(search_term, case=False, na=False)]

# If matches are found, calculate carbon footprint
if not matched_products.empty:
    print(f"Found {len(matched_products)} matches in Agribalyse for '{search_term}':")
    for _, product in matched_products.iterrows():
        product_name = product["Nom Français"]
        carbon_footprint_per_kg = product["Changement climatique"]
        weight_kg = 0.250  # Example: 250g
        carbon_footprint = carbon_footprint_per_kg * weight_kg
        print(f"- {product_name}: {carbon_footprint:.2f} kg CO₂e for 250g")
else:
    print(f"No matches found for '{search_term}' in Agribalyse.")


In [ ]:
!pip install unidecode
!pip install nltk

In [ ]:
import requests
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import unidecode
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
# Chemin vers le fichier Agribalyse
file_path = "../data/agribalyse-31-detail-par-ingredient.csv"
agribalyse_data = pd.read_csv(file_path)

def search_in_agribalyse_with_similarity(search_term, weight_kg):

    from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

    # Combine French and English stopwords
    french_stopwords = set(stopwords.words("french")) | ENGLISH_STOP_WORDS
    
    # Normalize data
    agribalyse_data["Nom Français"] = (
        agribalyse_data["Nom Français"].str.strip().str.lower().apply(unidecode.unidecode)
    )
    search_term = unidecode.unidecode(search_term.strip().lower())

    # Prepare data
    product_names = agribalyse_data["Nom Français"].dropna().tolist()
    product_names.append(search_term)

    # Vectorize
    stop_words_list = list(french_stopwords)

    # Vectorize
    vectorizer = TfidfVectorizer(analyzer="word", ngram_range=(1, 2), stop_words=stop_words_list)
    tfidf_matrix = vectorizer.fit_transform(product_names)

    # Compute cosine similarity
    similarities = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1])[0]
    best_match_index = similarities.argmax()
    best_match_score = similarities[best_match_index]
    best_match_product = agribalyse_data.iloc[best_match_index]

    # Debugging outputs
    print("Top 5 Similarity Scores and Matches:")
    top_indices = similarities.argsort()[-5:][::-1]
    for idx in top_indices:
        match_name = agribalyse_data.iloc[idx]["Nom Français"]
        score = similarities[idx]
        print(f" - {match_name}: {score:.2f}")

    if best_match_score > 0.15:  # Lower threshold
        product_name = best_match_product["Nom Français"]
        carbon_footprint_per_kg = best_match_product["Changement climatique"]
        carbon_footprint = carbon_footprint_per_kg * weight_kg
        print(f"- Closest match in Agribalyse: {product_name} (Similarity: {best_match_score:.2f})")
        print(f"  Carbon Footprint: {carbon_footprint:.2f} kg CO₂e for {weight_kg * 1000:.0f}g")
    else:
        print(f"No sufficiently similar products found for '{search_term}'.")



# Fonction pour chercher dans l'API Open Food Facts
def search_in_open_food_facts(product_name, weight_kg):
    search_url = "https://world.openfoodfacts.org/cgi/search.pl"
    params = {
        "search_terms": product_name,
        "search_simple": 1,
        "json": 1,
        "page_size": 1,
    }

    try:
        response = requests.get(search_url, params=params, timeout=5)  # Timeout de 5 secondes
        if response.status_code == 200:
            data = response.json()
            if data.get("products"):
                product = data["products"][0]  # Prendre le premier produit
                product_name = product.get("product_name", "Unknown Product")
                carbon_footprint_100g = product.get("carbon_footprint_100g", None)
                agribalyse_co2_total = product.get("agribalyse", {}).get("co2_total", None)
                
                print(f"Product Name: {product_name}")
                if carbon_footprint_100g:
                    carbon_footprint = (float(carbon_footprint_100g) / 100) * weight_kg * 1000
                    print(f"Carbon Footprint (per {weight_kg * 1000:.0f}g): {carbon_footprint:.2f} g CO₂e")
                elif agribalyse_co2_total:
                    carbon_footprint = float(agribalyse_co2_total) * weight_kg
                    print(f"Agribalyse Total CO₂ Emissions: {carbon_footprint:.2f} kg CO₂e")
                else:
                    print("Carbon Footprint data is not available in Open Food Facts.")
            else:
                print("No products found for your search term in Open Food Facts.")
                print("Switching to Agribalyse...")
                search_in_agribalyse_with_similarity(product_name, weight_kg)
        else:
            print(f"Failed to retrieve data from Open Food Facts. HTTP Status: {response.status_code}")
            print("Switching to Agribalyse...")
            search_in_agribalyse_with_similarity(product_name, weight_kg)
    except requests.exceptions.Timeout:
        print("Open Food Facts API timeout after 5 seconds. Switching to Agribalyse...")
        search_in_agribalyse_with_similarity(product_name, weight_kg)
    except Exception as e:
        print(f"An error occurred: {e}")
        search_in_agribalyse_with_similarity(product_name, weight_kg)

# Nom du produit et poids (exemple : 250g)
product_name = "muesli"
weight_kg = 0.250  # 250g

# Chercher dans Open Food Facts avec fallback sur Agribalyse
search_in_open_food_facts(product_name, weight_kg)


In [ ]:
import requests
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import unidecode

# Load Agribalyse datasets
datasets = {
    "par_etape": "../data/agribalyse-31-detail-par-etape.csv",
    "par_ingredient": "../data/agribalyse-31-detail-par-ingredient.csv",
    "synthese": "../data/agribalyse-31-synthese.csv"
}

# Define the correct column mappings
column_mappings = {
    "par_etape": "Nom du Produit en Français",
    "par_ingredient": "Nom Français",
    "synthese": "Nom du Produit en Français"
}

agribalyse_data = {key: pd.read_csv(path) for key, path in datasets.items()}


def normalize_text(text):
    """Normalize text by removing accents and converting to lowercase."""
    return unidecode.unidecode(text.strip().lower())


def filter_meaningful_components(components):
    """Filter out common stopwords and retain meaningful components."""
    french_stopwords = {"de", "et", "aux", "pour", "avec", "sur", "au"}  # Extend as needed
    return [component for component in components if component not in french_stopwords]


def search_in_dataset_with_similarity(dataset, column_name, search_term, weight_kg):
    """Search in a specific Agribalyse dataset with cosine similarity."""
    # Normalize data
    dataset[column_name] = dataset[column_name].str.strip().str.lower().apply(unidecode.unidecode)
    search_term = normalize_text(search_term)

    # Prepare data
    product_names = dataset[column_name].dropna().tolist()
    product_names.append(search_term)

    # Vectorize
    vectorizer = TfidfVectorizer(analyzer="word", ngram_range=(1, 2), stop_words="english")
    tfidf_matrix = vectorizer.fit_transform(product_names)

    # Compute cosine similarity
    similarities = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1])[0]
    best_match_index = similarities.argmax()
    best_match_score = similarities[best_match_index]
    best_match_product = dataset.iloc[best_match_index]

    # Debugging outputs
    print(f"Top 5 Similarity Scores and Matches in {column_name}:")
    top_indices = similarities.argsort()[-5:][::-1]
    for idx in top_indices:
        match_name = dataset.iloc[idx][column_name]
        score = similarities[idx]
        print(f" - {match_name}: {score:.2f}")

    if best_match_score > 0.2:  # Threshold for similarity
        product_name = best_match_product[column_name]
        carbon_footprint_per_kg = best_match_product["Changement climatique"]
        carbon_footprint = carbon_footprint_per_kg * weight_kg
        print(f"- Closest match in dataset: {product_name} (Similarity: {best_match_score:.2f})")
        print(f"  Carbon Footprint: {carbon_footprint:.2f} kg CO₂e for {weight_kg * 1000:.0f}g")
    else:
        print(f"No sufficiently similar products found for '{search_term}' in this dataset.")


def search_in_all_datasets(search_term, weight_kg):
    """Search across all Agribalyse datasets."""
    for key, dataset in agribalyse_data.items():
        column_name = column_mappings[key]
        print(f"\nSearching in dataset: {key.upper()}...")
        try:
            search_in_dataset_with_similarity(dataset, column_name, search_term, weight_kg)
        except KeyError:
            print(f"KeyError: The column '{column_name}' does not exist in the dataset {key}.")


def search_in_open_food_facts(product_name, weight_kg):
    """Search in Open Food Facts and fallback to Agribalyse if needed."""
    search_url = "https://world.openfoodfacts.org/cgi/search.pl"
    params = {
        "search_terms": product_name,
        "search_simple": 1,
        "json": 1,
        "page_size": 1,
    }

    try:
        response = requests.get(search_url, params=params, timeout=5)
        if response.status_code == 200:
            data = response.json()
            if data.get("products"):
                product = data["products"][0]
                product_name = product.get("product_name", "Unknown Product")
                carbon_footprint_100g = product.get("carbon_footprint_100g", None)

                print(f"Product Name: {product_name}")
                if carbon_footprint_100g:
                    carbon_footprint = (float(carbon_footprint_100g) / 100) * weight_kg * 1000
                    print(f"Carbon Footprint (per {weight_kg * 1000:.0f}g): {carbon_footprint:.2f} g CO₂e")
                else:
                    print("Carbon Footprint data is not available in Open Food Facts.")
                    print("Switching to Agribalyse with all datasets...")
                    search_in_all_datasets(product_name, weight_kg)
            else:
                print("No products found for your search term in Open Food Facts.")
                print("Switching to Agribalyse...")
                search_in_all_datasets(product_name, weight_kg)
        else:
            print(f"Failed to retrieve data from Open Food Facts. HTTP Status: {response.status_code}")
            print("Switching to Agribalyse...")
            search_in_all_datasets(product_name, weight_kg)
    except requests.exceptions.Timeout:
        print("Open Food Facts API timeout after 5 seconds. Switching to Agribalyse...")
        search_in_all_datasets(product_name, weight_kg)
    except Exception as e:
        print(f"An error occurred: {e}")
        search_in_all_datasets(product_name, weight_kg)


# Test Input
product_name = "soda à l'orange"
weight_kg = 1  # 1 kg
search_in_open_food_facts(product_name, weight_kg)


In [65]:
import requests
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import unidecode
import os
import json

# Load Agribalyse datasets
datasets = {
    "par_etape": "../data/agribalyse-31-detail-par-etape.csv",
    "par_ingredient": "../data/agribalyse-31-detail-par-ingredient.csv",
    "synthese": "../data/agribalyse-31-synthese.csv"
}

# Define the correct column mappings
column_mappings = {
    "par_etape": "Nom du Produit en Français",
    "par_ingredient": "Nom Français",
    "synthese": "Nom du Produit en Français"
}

# Load datasets into memory
agribalyse_data = {key: pd.read_csv(path) for key, path in datasets.items()}

# Pre-define vectorizer
vectorizer = TfidfVectorizer(analyzer="word", ngram_range=(1, 2), stop_words="english")

# Caching setup
CACHE_FILE = "open_food_facts_cache.json"
if os.path.exists(CACHE_FILE):
    with open(CACHE_FILE, "r") as f:
        cache = json.load(f)
else:
    cache = {}


def save_cache():
    """Save cache to file."""
    with open(CACHE_FILE, "w") as f:
        json.dump(cache, f)


def normalize_text(text):
    """Normalize text by removing accents and converting to lowercase."""
    return unidecode.unidecode(text.strip().lower())


def filter_meaningful_components(components):
    """Filter out common stopwords and retain meaningful components."""
    french_stopwords = {"de", "et", "aux", "pour", "avec", "sur", "au"}  # Extend as needed
    return [component for component in components if component not in french_stopwords]


def search_in_dataset_with_similarity(dataset, column_name, search_term, weight_kg):
    """Search in a specific Agribalyse dataset with cosine similarity."""
    # Copy and normalize the dataset column
    normalized_column = dataset[column_name].dropna().str.strip().str.lower().apply(unidecode.unidecode)
    search_term = normalize_text(search_term)

    # Prepare data for vectorization
    product_names = normalized_column.tolist()
    product_names.append(search_term)

    # Vectorize and calculate cosine similarity
    tfidf_matrix = vectorizer.fit_transform(product_names)
    similarities = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1])[0]

    # Get the best match
    best_match_index = similarities.argmax()
    best_match_score = similarities[best_match_index]

    # Handle results
    if best_match_score > 0.2:  # Threshold for similarity
        best_match_product = dataset.iloc[best_match_index]
        product_name = best_match_product[column_name]
        carbon_footprint_per_kg = best_match_product.get("Changement climatique", 0)
        carbon_footprint = carbon_footprint_per_kg * weight_kg
        return {"product_name": product_name, "carbon_footprint": carbon_footprint}

    return None


def search_in_all_datasets(search_term, weight_kg):
    """Search across all Agribalyse datasets."""
    results = []
    for key, dataset in agribalyse_data.items():
        column_name = column_mappings[key]
        try:
            result = search_in_dataset_with_similarity(dataset, column_name, search_term, weight_kg)
            if result:
                results.append(result)
        except KeyError:
            print(f"KeyError: The column '{column_name}' does not exist in the dataset {key}.")
    return results


def search_in_open_food_facts(product_name, weight_kg):
    """Search in Open Food Facts and fallback to Agribalyse if needed."""
    # Check cache first
    if product_name in cache:
        print(f"Using cached data for '{product_name}'...")
        return cache[product_name]

    # API Request
    search_url = "https://world.openfoodfacts.org/cgi/search.pl"
    params = {
        "search_terms": product_name,
        "search_simple": 1,
        "json": 1,
        "page_size": 1,
    }

    try:
        response = requests.get(search_url, params=params, timeout=5)
        if response.status_code == 200:
            data = response.json()
            if data.get("products"):
                product = data["products"][0]
                product_name = product.get("product_name", "Unknown Product")
                carbon_footprint_100g = product.get("carbon_footprint_100g", None)

                if carbon_footprint_100g:
                    carbon_footprint = (float(carbon_footprint_100g) / 100) * weight_kg * 1000
                    result = {"product_name": product_name, "carbon_footprint": carbon_footprint}
                    cache[product_name] = result
                    save_cache()
                    return result
                else:
                    print("Carbon Footprint data is not available in Open Food Facts.")

        print("No products found in Open Food Facts.")
    except requests.exceptions.Timeout:
        print("Open Food Facts API timeout after 5 seconds.")
    except Exception as e:
        print(f"An error occurred: {e}")

    # Fallback to Agribalyse
    print("Switching to Agribalyse with all datasets...")
    components = filter_meaningful_components(product_name.split())
    results = []
    for component in components:
        print(f"Searching for component: {component}...")
        component_results = search_in_all_datasets(component, weight_kg)
        results.extend(component_results)

    if results:
        # Calculate the mean carbon footprint
        total_footprint = sum(r["carbon_footprint"] for r in results)
        mean_footprint = total_footprint / len(results)
        final_result = {"product_name": product_name, "carbon_footprint": mean_footprint}
        cache[product_name] = final_result
        save_cache()
        return final_result

    return {"product_name": product_name, "carbon_footprint": None}


# Test Input
product_name = "soda à l'orange"
weight_kg = 0.5  # 1 kg
result = search_in_open_food_facts(product_name, weight_kg)
if result["carbon_footprint"] is not None:
    print(f"Final Result: {result['product_name']} - {result['carbon_footprint']:.2f} kg CO₂e")
else:
    print(f"No carbon footprint data available for '{product_name}'.")


Carbon Footprint data is not available in Open Food Facts.
No products found in Open Food Facts.
Switching to Agribalyse with all datasets...
Searching for component: Fanta...
Searching for component: Orange...
Searching for component: 2ltr...
Final Result: Fanta Orange 2ltr - 0.12 kg CO₂e


In [ ]:
import requests
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import unidecode
import os
import json
from concurrent.futures import ThreadPoolExecutor
import logging

# Logging setup
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s]: %(message)s")

# Load Agribalyse datasets
datasets = {
    "par_etape": "../data/agribalyse-31-detail-par-etape.csv",
    "par_ingredient": "../data/agribalyse-31-detail-par-ingredient.csv",
    "synthese": "../data/agribalyse-31-synthese.csv"
}

column_mappings = {
    "par_etape": "Nom du Produit en Français",
    "par_ingredient": "Nom Français",
    "synthese": "Nom du Produit en Français"
}

agribalyse_data = {key: pd.read_csv(path) for key, path in datasets.items()}

# Precompute TF-IDF for datasets
vectorizer = TfidfVectorizer(analyzer="word", ngram_range=(1, 2), stop_words="english")
vocabulary = None
precomputed_tfidf = {}

for key, dataset in agribalyse_data.items():
    column_name = column_mappings[key]
    dataset[column_name] = dataset[column_name].dropna().str.strip().str.lower().apply(unidecode.unidecode)
    
    if vocabulary is None:
        # Fit the vectorizer on the first dataset and save the vocabulary
        tfidf_matrix = vectorizer.fit_transform(dataset[column_name].tolist())
        vocabulary = vectorizer.vocabulary_
    else:
        # Use the same vocabulary for other datasets
        temp_vectorizer = TfidfVectorizer(analyzer="word", ngram_range=(1, 2), stop_words="english", vocabulary=vocabulary)
        tfidf_matrix = temp_vectorizer.fit_transform(dataset[column_name].tolist())
    
    precomputed_tfidf[key] = {"matrix": tfidf_matrix, "names": dataset[column_name]}

# Caching setup
CACHE_FILE = "open_food_facts_cache.json"
if os.path.exists(CACHE_FILE):
    with open(CACHE_FILE, "r") as f:
        cache = json.load(f)
else:
    cache = {}


def save_cache():
    """Save cache to file."""
    with open(CACHE_FILE, "w") as f:
        json.dump(cache, f)


def normalize_text(text):
    """Normalize text by removing accents and converting to lowercase."""
    return unidecode.unidecode(text.strip().lower())


def filter_meaningful_components(components):
    """Filter out common stopwords and retain meaningful components."""
    french_stopwords = {"de", "et", "aux", "pour", "avec", "sur", "au"}
    return [component for component in components if component not in french_stopwords]


def search_in_dataset_with_similarity(dataset_key, search_term, weight_kg):
    """Search in a specific Agribalyse dataset using precomputed TF-IDF."""
    column_name = column_mappings[dataset_key]
    tfidf_data = precomputed_tfidf[dataset_key]
    tfidf_matrix = tfidf_data["matrix"]
    product_names = tfidf_data["names"]

    search_term_vector = vectorizer.transform([normalize_text(search_term)])
    similarities = cosine_similarity(search_term_vector, tfidf_matrix)[0]

    best_match_index = similarities.argmax()
    best_match_score = similarities[best_match_index]

    if best_match_score > 0.2:
        best_match_product = agribalyse_data[dataset_key].iloc[best_match_index]
        product_name = best_match_product[column_name]
        carbon_footprint_per_kg = best_match_product.get("Changement climatique", 0)
        carbon_footprint = carbon_footprint_per_kg * weight_kg
        return {"product_name": product_name, "carbon_footprint": carbon_footprint}

    return None


def search_in_all_datasets(search_term, weight_kg):
    """Search across all Agribalyse datasets."""
    results = []
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(search_in_dataset_with_similarity, key, search_term, weight_kg) for key in agribalyse_data]
        for future in futures:
            result = future.result()
            if result:
                results.append(result)
    return results


def search_in_open_food_facts(product_name, weight_kg):
    """Search in Open Food Facts and fallback to Agribalyse."""
    if product_name in cache:
        logging.info(f"Using cached data for '{product_name}'...")
        return cache[product_name]

    search_url = "https://world.openfoodfacts.org/cgi/search.pl"
    params = {
        "search_terms": product_name,
        "search_simple": 1,
        "json": 1,
        "page_size": 1,
    }

    try:
        response = requests.get(search_url, params=params, timeout=5)
        if response.status_code == 200:
            data = response.json()
            if data.get("products"):
                product = data["products"][0]
                product_name = product.get("product_name", "Unknown Product")
                carbon_footprint_100g = product.get("carbon_footprint_100g", None)

                if carbon_footprint_100g:
                    carbon_footprint = (float(carbon_footprint_100g) / 100) * weight_kg * 1000
                    result = {"product_name": product_name, "carbon_footprint": carbon_footprint}
                    cache[product_name] = result
                    save_cache()
                    return result
        logging.info("No products found in Open Food Facts.")
    except requests.exceptions.Timeout:
        logging.warning("Open Food Facts API timeout.")
    except Exception as e:
        logging.error(f"Error occurred: {e}")

    # Fallback to Agribalyse
    logging.info("Switching to Agribalyse datasets...")
    components = filter_meaningful_components(product_name.split())
    results = []
    for component in components:
        logging.info(f"Searching for component: {component}...")
        component_results = search_in_all_datasets(component, weight_kg)
        results.extend(component_results)

    if results:
        total_footprint = sum(r["carbon_footprint"] for r in results)
        mean_footprint = total_footprint / len(results)
        final_result = {"product_name": product_name, "carbon_footprint": mean_footprint}
        cache[product_name] = final_result
        save_cache()
        return final_result

    return {"product_name": product_name, "carbon_footprint": None}


# Test Input
product_name = "pomme de terre"
weight_kg = 0.5
result = search_in_open_food_facts(product_name, weight_kg)
if result["carbon_footprint"] is not None:
    logging.info(f"Final Result: {result['product_name']} - {result['carbon_footprint']:.2f} kg CO₂e")
else:
    logging.info(f"No carbon footprint data available for '{product_name}'.")


2025-01-20 20:13:31,372 [INFO]: No products found in Open Food Facts.
2025-01-20 20:13:31,374 [INFO]: Switching to Agribalyse datasets...
2025-01-20 20:13:31,375 [INFO]: Searching for component: Chips...
2025-01-20 20:13:31,383 [INFO]: Searching for component: saveur...
2025-01-20 20:13:31,387 [INFO]: Searching for component: Poulet...
2025-01-20 20:13:31,390 [INFO]: Searching for component: Braisé...
2025-01-20 20:13:31,395 [INFO]: Final Result: Chips saveur Poulet Braisé - 2.64 kg CO₂e
